In [1]:
from pynq import Overlay
from pynq import allocate
import matplotlib.pyplot as plt
import numpy as np

In [2]:
overlay = Overlay('/home/xilinx/pynq/overlays/multiplier/axis_multiplier.bit')

In [3]:
import pynq.lib.dma

dma = overlay.axi_dma_0

In [4]:
#Class copied from the original script. You can easily import it as module without copying the code.
from fxpmath import Fxp
import numpy as np

class Converter():

    def forward_conversion(self, input_data, signed, total_bits, fractional_bits):
        '''
        Converts input data from float/int python data types to ap_fixed with total bits and fractional_bits and returns its uint32 equivalent
        :param input_data: can be both a single int/float number or a numpy array
        :param signed: Boolean, if the input data is signed or not
        :param total_bits: numer of total bits used to represent the input data
        :param fractional_bits: number of fractional bits used to represent the input data. Integer bits = total bits - fractional bits
        :return: input data converted to uint32 format. 0.5 can be represented with 4 bits as 0.100. This is converted into 0100 (fractional point removed) and then converted to int.
                 0.5 as input is converted to 4 as uint32.
        '''
        fixed_point_representation = Fxp(input_data, signed=signed, n_word = total_bits, n_frac = fractional_bits)
        uint_coverted = np.uint32(fixed_point_representation.uraw())
        return uint_coverted



    def backward_conversion(self, input_data, total_bits, fractional_bits):
        '''
        Converts input data from uint32 format to float with total_bits and fractional_bits resolution
        :param input_data: can be both a single int/float number or a numpy array
        :param total_bits: numer of total bits used to represent the input data
        :param fractional_bits: number of fractional bits used to represent the input data. Integer bits = total bits - fractional bits
        :return: converted input data from uint32 to float
        '''

        if type(input_data) is not np.ndarray:
            input_data = np.array([input_data])

        #Function taken from here: https://discuss.pynq.io/t/how-to-use-ap-fixed-data-type-to-communicate-with-the-ip-made-by-the-vivado-hls/679/5
        condition = 1 << (total_bits - 1)
        mask = (~((1 << total_bits) - 1)) & 0xFFFFFFFF
        return np.where(input_data < condition, input_data, (input_data.view('u4') | mask).view('i4')) / (1 << fractional_bits)

In [5]:
input_data = np.array([1,0.25,-0.5])
converter = Converter()
#NB: in the total_bits-Fractional_bits declaration, remembr to match the actual data size of your hardware ip!
input_data_forward_conversion = converter.forward_conversion(input_data=input_data, signed=True, total_bits = 28, fractional_bits=22)
input_data_forward_conversion

/usr/local/lib/python3.6/dist-packages/fxpmath/objects.py:579: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  if original_vdtype != complex and not np.issubdtype(original_vdtype, complex):


array([  4194304,   1048576, 266338304], dtype=uint32)

In [6]:
from pynq import allocate
import numpy as np

# IGNORE THIS COMMENT
#IMPORTANTE: con l'output nel design a ap_fixed<32> funziona correttamente. Con delle dimensioni inferiori, spara valori a caso.
# molto probabilmente è un problema di alignment e/o estensione dei bit. Ad esempio con l'output a 24 bit, l'adapter deve estendere
# a 32 bit per la DMA, forse è qui il problema.

# Dalla documentazione (https://www.xilinx.com/support/documentation/ip_documentation/axis_infrastructure_ip_suite/v1_1/pg085-axi4stream-infrastructure.pdf)
# il data width converter se scrivo da più piccolo a più grande, mi mette assieme più chunk e poi scrive tutto assieme,
# viceversa mette in serie (è scritto nelle prime righe dela documentazione)
# Quindi con l'output non multiplo di 32 (ad esempio 24), si spacca tutto. In teoria il problema è che l'IP scrive come risultato
# 24 bit, tuttavia l'input della DMA e il data converter è a 32 bit, quindi mi concatena 8 bit del risultato successivo ai 24 del precedente
# e quindi mi vengono risultati a caso

in_buffer = allocate(shape=(3,), dtype=np.uint32)
out_buffer = allocate(shape=(1,), dtype=np.uint32)

# The following assignment makes the kernel die, don't know why
#in_buffer[:] = input_data_forward_conversion[:]

#The following one works fine
for i in range(3):
    in_buffer[i] = input_data_forward_conversion[i]
    


dma.recvchannel.transfer(out_buffer)
dma.sendchannel.transfer(in_buffer)

dma.sendchannel.wait()
dma.recvchannel.wait()

out_buffer

PynqBuffer([3145728], dtype=uint32)

In [7]:
result = converter.backward_conversion(out_buffer, 28, 22)
result

array([[0.75]])